# Analysis of the attention matrices of Dual Attention Recurrent Neural Networks

06/2021 Jonathan Fiorentino

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import antropy as ant
import os
import pandas as pd
import re

# Load the data for each gene regulatory network

In [ ]:
noiseFolder='./DATA/Noise_analysis/'
subfolders=[x[0] for x in os.walk('./DATA/')]

stringList=['FullyConnected','FullyRepressed','MasterRegulator','mediumConnection','SparseConnection',
         'Oscillating','ExternalSignal']

netList=[]
mseMatList=[]
mapeMatList=[]
zeroGenesList=[]
permEntlist=[]

for string in stringList:
    subfold = [s for s in subfolders if string in s]
    files=[s for s in os.listdir(noiseFolder) if string in s and 'txt' in s and ('mse' in s or 'mape' in s)]
    if len(files)==2:
        timeseries=[f for f in os.listdir(subfold[0]) if 'Protein' in f and 'txt' in f][0]
        timeseriesdata=np.loadtxt(subfold[0]+'/'+timeseries)
        timeseriesdata=timeseriesdata[:,1:]
        permentonenet=[]
        for i in range(timeseriesdata.shape[1]):
            perment=ant.perm_entropy(timeseriesdata[:,i], normalize=True)
            permentonenet.append(perment)
        permEntlist.append(permentonenet)
        zeroGenesList.append(np.where(np.all(timeseriesdata==0,axis=0))[0])
        netList.append(string)
        for file in files:
            if 'mse' in file:
                mseMat=np.loadtxt(noiseFolder+file)
                mseMatList.append(mseMat)
            if 'mape' in file:
                mapeMat=np.loadtxt(noiseFolder+file)
                mapeMatList.append(mapeMat)
    else:
        tmpfiles=[s for s in files if 'mse' in s]
        substrings=[re.sub(r'^.*?nr', 'nr', s) for s in tmpfiles]
        substrings=[s.replace('.txt','') for s in substrings]
        numbers=[int(re.sub(r'^.*?na', '', s)) for s in substrings]
        numbers_sorted=sorted(range(len(numbers)), key=lambda k: numbers[k])
        substrings=[substrings[n] for n in numbers_sorted]
        for sub in substrings:
            timeseries=[f for f in os.listdir(subfold[0]) if 'Protein' in f and 'txt' in f and sub in f][0]
            timeseriesdata=np.loadtxt(subfold[0]+'/'+timeseries)
            timeseriesdata=timeseriesdata[:,1:]
            permentonenet=[]
            for i in range(timeseriesdata.shape[1]):
                perment=ant.perm_entropy(timeseriesdata[:,i], normalize=True)
                permentonenet.append(perment)
                permEntlist.append(permentonenet)
            zeroGenesList.append(np.where(np.all(timeseriesdata==0,axis=0))[0])
            netList.append(string+'_'+sub)
            newfiles=[s for s in files if sub in s]
            for file in newfiles:
                if 'mse' in file:
                    mseMat=np.loadtxt(noiseFolder+file)
                    mseMatList.append(mseMat)
                if 'mape' in file:
                    mapeMat=np.loadtxt(noiseFolder+file)
                    mapeMatList.append(mapeMat)

In [ ]:
# For each network, build dataframes of the MSE and MAPE for each gene and each noise level
# Compute the mean and the CV of MSE and MAPE over the different noise levels
# Remove genes that are never expressed
# Remove genes for which the MSE of the original prediction is zero (those are genes that are expressed only in
# a subset of the time window used for training and zero elsewhere)
import scipy
def todf(arr):
    df=pd.DataFrame(data=arr)
    df.index=['Gene_'+str(i) for i in range(20)]
    
    return df;

msedfList=[]
mapedfList=[]
rankedmeanMSElist=[]
rankedcvMSElist=[]
rankedmeanMAPElist=[]
rankedcvMAPElist=[]
i=0

NetNoiseMat=np.zeros((len(netList),40))

for (mat1,mat2,zg) in zip(mseMatList,mapeMatList,zeroGenesList):
    print(netList[i])
    mydf=todf(mat1)
    if len(zg)>0:
        genes_to_delete=['Gene_'+str(j) for j in zg]
        mydf=mydf.drop(genes_to_delete)
    
    mydf=mydf[mydf[mydf.columns[0]] != 0]
    mydf['meanMSE']=todf(mat1).mean(axis=1)
    mydf['cvMSE']=todf(mat1).std(axis=1)/todf(mat1).mean(axis=1)
    
    # Uncomment this for independent sorting of means and coefficient of variations
#     meanlist=sorted(list(todf(mat1).mean(axis=1)))
#     meanlist=[x/max(meanlist) for x in meanlist]
#     varlist=sorted(list(todf(mat1).std(axis=1)/todf(mat1).mean(axis=1)))
#     varlist=[0 if np.isnan(x) else x for x in varlist]
    
    # CV normalization
#     varlist=[x/max(varlist) for x in varlist]
    
    # Uncomment this for dependent sorting of means and coefficient of variations
    meanlist=list(todf(mat1).mean(axis=1))
    indices_sorted=sorted(range(len(meanlist)), key=lambda k: meanlist[k])
    meanlist=[meanlist[k] for k in indices_sorted]
    meanlist=[x/max(meanlist) for x in meanlist]
#     varlist=list(todf(mat1).std(axis=1)/todf(mat1).mean(axis=1))
    varlist=list(todf(mat1).var(axis=1))
    varlist=[varlist[k] for k in indices_sorted]
    varlist=[0 if np.isnan(x) else x for x in varlist]
    
    # CV normalization
    varlist=[x/max(varlist) for x in varlist]
    
    if len(meanlist)<20:
        meanlist=[0.]*(20-len(meanlist))+meanlist
        varlist=[0.]*(20-len(varlist))+varlist
    NetNoiseMat[i]=np.array(meanlist+varlist)
    print(len(meanlist),len(varlist))
#     print(scipy.stats.spearmanr(mydf['meanMSE'],mydf['cvMSE']))
    msedfList.append(mydf)
    rankedmeanMSEdf=mydf.sort_values('meanMSE')
    rankedcvMSEdf=mydf.sort_values('cvMSE')
    rankedmeanMSElist.append(rankedmeanMSEdf)
    rankedcvMSElist.append(rankedcvMSEdf)
    mydf2=todf(mat2)
    if len(zg)>0:
        genes_to_delete=['Gene_'+str(j) for j in zg]
        mydf2=mydf2.drop(genes_to_delete)
    mydf2=mydf2[mydf2[mydf2.columns[0]] != 0]
    mydf2['meanMAPE']=todf(mat2).mean(axis=1)
    mydf2['cvMAPE']=todf(mat2).std(axis=1)/todf(mat2).mean(axis=1)
#     print(scipy.stats.spearmanr(mydf2['meanMAPE'],mydf2['cvMAPE']))
    mapedfList.append(mydf2)
    rankedmeanMAPEdf=mydf2.sort_values('meanMAPE')
    rankedcvMAPEdf=mydf2.sort_values('cvMAPE')
    rankedmeanMAPElist.append(rankedmeanMAPEdf)
    rankedcvMAPElist.append(rankedcvMAPEdf)
    i+=1

# Clustering the matrix of mean and variances of the MSE

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import linkage
from sklearn.metrics import silhouette_score
from sklearn.metrics import calinski_harabasz_score
from sklearn.metrics import davies_bouldin_score
from matplotlib.gridspec import GridSpec
from matplotlib.pyplot import cm
from scipy.cluster import hierarchy
import matplotlib as mpl
import rpy2.robjects.numpy2ri
rpy2.robjects.numpy2ri.activate()
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr


mycolors=['C0','C1','C2','C3','C4','C5','C6','C7','C8','C9','C10','C11','C12']

markers=['o', 'v', '^', '<', '>', '8', 's', 'p', '*', 'h', 'H', 'D']

def getNewick(node, newick, parentdist, leaf_names):
    if node.is_leaf():
        return "%s:%.2f%s" % (leaf_names[node.id], parentdist - node.dist, newick)
    else:
        if len(newick) > 0:
            newick = "):%.2f%s" % (parentdist - node.dist, newick)
        else:
            newick = ");"
        newick = getNewick(node.get_left(), newick, node.dist, leaf_names)
        newick = getNewick(node.get_right(), ",%s" % (newick), node.dist, leaf_names)
        newick = "(%s" % (newick)
        return newick

# Function that plots a dendrogram
def plot_dendrogram(model, **kwargs):

    # Children of hierarchical clustering
    children = model.children_

    # Distances between each pair of children
    # Since we don't have this information, we can use a uniform one for plotting
    distance = np.arange(children.shape[0])

    # The number of observations contained in each cluster level
    no_of_observations = np.arange(2, children.shape[0]+2)

    # Create linkage matrix and then plot the dendrogram
    linkage_matrix = np.column_stack([children, distance, no_of_observations]).astype(float)
    

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs,color_threshold=0)

def PlotResults(model,netList,mydf,label,save=False):
    
    fig=plt.figure(figsize=(10,10))
    fig.suptitle(label)
    gs=GridSpec(2,2) # 2 rows, 2 columns
    
    ax1=fig.add_subplot(gs[0,:]) # Second row, span all columns
    ax2=fig.add_subplot(gs[1,0]) # First row, first column
    ax3=fig.add_subplot(gs[1,1]) # First row, second column
    ax1.text(-0.1, 1.15, 'A', transform=ax1.transAxes,
      fontsize=20, fontweight='bold', va='top', ha='right')
    ax2.text(-0.1, 1.15, 'B', transform=ax2.transAxes,
      fontsize=20, fontweight='bold', va='top', ha='right')
    ax3.text(-0.1, 1.15, 'C', transform=ax3.transAxes,
      fontsize=20, fontweight='bold', va='top', ha='right')
    # Plot the dendrogram
    
    plot_dendrogram(model, labels=netList,leaf_rotation=90,ax=ax1)
    sns.scatterplot(x='principal component 1',
                y='principal component 2',s=100,
                data=mydf,hue='networks',style='networks',markers=markers,ax=ax2)
    ax2.legend(frameon=False,bbox_to_anchor=(1,1),title='Networks')
    
    sns.scatterplot(x='principal component 1',
                y='principal component 2',s=100,
                data=mydf,hue='clusters',style='networks',markers=markers,palette='deep',ax=ax3)
    N=model.n_clusters+1
    handles, labels = ax3.get_legend_handles_labels()
    handles=handles[:N]
    labels=labels[:N]
    ax3.legend(handles, labels,frameon=True,loc=0)
    
    fig.tight_layout()
    
    if save==True:
        plt.savefig('Clustering_'+label+'.pdf',bbox_inches='tight'),plt.close()
    else:
        plt.show(),plt.close()


# Determine optimal number of clusters
def optimalClusters(X):

    sil_score_max = -1 #this is the minimum possible score
    for n_clusters in range(2,10):
        model = AgglomerativeClustering(n_clusters=n_clusters, affinity='euclidean', linkage='complete')
        labels = model.fit_predict(X)
        sil_score = silhouette_score(X, labels)

        print("The average silhouette score for %i clusters is %0.2f" %(n_clusters,sil_score))
        if sil_score > sil_score_max:
            sil_score_max = sil_score
            best_n_clusters = n_clusters
    return best_n_clusters;

# Function that takes a matrix, performs hierarchical clustering
# and chooses the optimal number of clusters
# Then it plots a dendrogram and a PCA plot with the nets and the results of the clustering
def Cluster_and_PCA(mat,netList, label, scale=False, save=False):
    if scale==True:
        x = StandardScaler().fit_transform(mat)
    else:
        x=mat
    # Principal Components Analysis
    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(x)
    principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])
    
    
    # Find optimal number of clusters
    best_n_clusters = optimalClusters(x)
    # Hierarchical clustering
    cluster = AgglomerativeClustering(n_clusters=best_n_clusters, affinity='euclidean', linkage='complete')
        
    
    
    myclusters=cluster.fit_predict(x)
    principalDf['networks']=netList
    principalDf['clusters']=myclusters
    model = cluster.fit(x)

    PlotResults(model,netList,principalDf,label,save=save)
    
    Z=linkage(x, method='complete', metric='euclidean', optimal_ordering=False)
    leaf_names=dendrogram(Z, labels=netList,leaf_rotation=90,color_threshold=0)['ivl']
    tree = hierarchy.to_tree(Z)
    
    newick_tree=getNewick(tree, "", tree.dist, leaf_names)
    
    return newick_tree,model.labels_;

In [ ]:
NetNoiseMatdf=pd.DataFrame(data=NetNoiseMat[:,:20])
NetNoiseMatdf.index=netList
NetNoiseMatdf=NetNoiseMatdf[NetNoiseMatdf.columns[::-1]]
NetNoiseMatdf.to_csv('noisematrix.csv')

In [ ]:
labelList=[]
treeList=[]
clustersList=[]

noisetree,labnoise=Cluster_and_PCA(NetNoiseMat,netList, 'Noise_analysis', scale=True,save=False)

treeList.append(noisetree)
clustersList.append(labnoise)
labelList.append('Noise_analysis')

In [ ]:
x = StandardScaler().fit_transform(NetNoiseMat)
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])
    
    
# Find optimal number of clusters
best_n_clusters = optimalClusters(x)
# Hierarchical clustering
cluster = AgglomerativeClustering(n_clusters=best_n_clusters, affinity='euclidean', linkage='complete')
myclusters=cluster.fit_predict(x)
principalDf['networks']=netList
principalDf['clusters']=myclusters
model = cluster.fit(x)



In [ ]:
newindex=['ExternalSignal_nr10_na10', 'ExternalSignal_nr1_na1',
       'ExternalSignal_nr5_na5', 'FullyConnected',
       'FullyRepressed',
       'Oscillating_nr10_na10',
       'Oscillating_nr15_na15',
       'Oscillating_nr1_na1',
       'Oscillating_nr5_na5',
       'SparseConnection', 'mediumConnection',
       'MasterRegulator']

# Clustering the graph descriptors matrices of the attention layer

In [ ]:
for file in os.listdir('./DATA/Matrici_Edo/'):
    mydf=pd.read_csv('./DATA/Matrici_Edo/'+file,skiprows=1,index_col=0,header=None)
    mydf.index=newindex
    mydf=mydf.reindex(netList)
    
    label=file.replace('.csv','')
    label=label.replace('df_','')
    print(file,label)
    mydf.to_csv('matrix_'+label+'.csv')
    d,labs=Cluster_and_PCA(mydf,list(mydf.index), label, scale=True,save=False)
    treeList.append(d)
    labelList.append(label)
    clustersList.append(labs)


# Downstream comparisons between the clusterings

In [ ]:
import igraph

In [ ]:
for i in range(len(treeList)):
    print(labelList[i])
    text_file = open(labelList[i]+'_tree.txt', 'w')
    #write string to file
    text_file.write(treeList[i]) 
    #close file
    text_file.close()

In [ ]:
from ete3 import Tree

VImatrix=np.zeros((len(treeList),len(treeList)))

for i in range(len(treeList)):
    for j in range(len(treeList)):
        
        cl1 = igraph.clustering.Clustering(clustersList[i])
        cl2=igraph.clustering.Clustering(clustersList[j])
        
        VImatrix[i,j]=igraph.compare_communities(cl1, cl2, method='vi', remove_none=False)

In [ ]:
indices=[0,1,3,7]

VImatrixnew=VImatrix[indices,:]
VImatrixnew=VImatrixnew[:,indices]

labelListnew=['Noise_analysis',
 'HubScore','Betweenness','Clustering_Coefficient']

In [ ]:
fig,ax=plt.subplots()

mask = np.triu(np.ones_like(VImatrixnew, dtype=bool))

sns.heatmap(VImatrixnew,cmap='viridis',xticklabels=labelListnew,yticklabels=labelListnew,
           cbar_kws={'label': 'Variation of information (bits)'},mask=mask,annot=True,ax=ax)

plt.savefig('vimatrixnew.pdf',bbox_inches='tight'),plt.close()

In [ ]:
dendromat=pd.read_csv('dendrodist.csv',index_col=0)
dendromatnew=dendromat.loc[:,labelListnew]
dendromatnew=dendromatnew.loc[labelListnew,:]

fig,ax=plt.subplots()

mask = np.triu(np.ones_like(dendromatnew, dtype=bool))

sns.heatmap(dendromatnew,cmap='viridis',xticklabels=labelListnew,yticklabels=labelListnew,
           cbar_kws={'label': 'Tree distance'},mask=mask,annot=True,ax=ax)

plt.savefig('rfdistnew.pdf',bbox_inches='tight'),plt.close()

# Scatter plots of correlation between matrices 

In [ ]:
tmp_NetNoiseMat=NetNoiseMat[:,:20]
tmp_NetNoiseMat=tmp_NetNoiseMat[:,::-1]

myMatList=[]
myMatList.append(tmp_NetNoiseMat)

for file in os.listdir('./DATA/Matrici_Edo/'):
    mydf=pd.read_csv('./DATA/Matrici_Edo/'+file,skiprows=1,index_col=0,header=None)
    mydf.index=newindex
    mydf=mydf.reindex(netList)
    label=file.replace('.csv','')
    label=label.replace('df_','')
    
    if label=='HubScore' or label=='Betweenness' or label=='Clustering_Coefficient':
        myMatList.append(np.array(mydf))


In [ ]:
myproplist=['Normalized MSE from noise analysis','Hub score','Betweenness','Clustering coefficient']
fig,ax=plt.subplots(2,3,figsize=(10,5))
pear=pd.read_csv('pearson_logmatrix.csv',index_col=0)
k=0
l=0
for i in range(len(myMatList)):
    for j in range(len(myMatList)):
        if i!=j and j>i:
            print(i,j,k,l)

            ax[k,l].set_xlabel(myproplist[i])
            ax[k,l].set_ylabel(myproplist[j])
            ax[k,l].set_title(r'$\rho=%.2f$' % (pear.iloc[i,j]))
            ax[k,l].scatter(np.sign(myMatList[i])*np.log(1.0+np.abs(myMatList[i])),
                                                         np.sign(myMatList[j])*np.log(1.0+np.abs(myMatList[j])))
            l+=1
            if l==3:
                k=1
                l=0
fig.tight_layout()
plt.savefig('cmp_matrices.jpg',bbox_inches='tight'),plt.close()

# Heatmaps of MSE matrices ranked by MSE mean or CV

In [ ]:
fig,ax=plt.subplots(1,2,figsize=(10,4))
xticks = ['0','0.1','0.2','0.3','0.4','0.5','0.6','0.7','0.8','0.9','mean']

for i in range(len(netList)):
    if netList[i]=='Oscillating_nr10_na10':
        sns.heatmap(rankedmeanMSElist[i].drop(columns='cvMSE'),cmap='viridis',ax=ax[0],
                xticklabels=xticks,yticklabels=rankedmeanMSElist[i].index,
                cbar_kws={'label': 'Mean Squared Error'})
        ax[0].set_title('MSE '+netList[i])
    if netList[i]=='MasterRegulator':
        sns.heatmap(rankedmeanMSElist[i].drop(columns='cvMSE'),cmap='viridis',ax=ax[1],
                xticklabels=xticks,yticklabels=rankedmeanMSElist[i].index,
                cbar_kws={'label': 'Mean Squared Error'})
        ax[1].set_title('MSE '+netList[i])


ax[0].set_xlabel(r'$\sigma$')
ax[1].set_xlabel(r'$\sigma$')

fig.tight_layout()
plt.savefig('Noise_matrices.pdf'),plt.close()